# Zadania do wykonania na danych w Pandas

1. **Załaduj DataFrame do Pandas, posortuj według `timestamp`, zmień indeks na kolumnę `timestamp` oraz usuń kolumnę `index`.**
   - *Wskazówka*: Użyj `pd.read_csv()` do załadowania danych oraz `sort_values()`,`set_index()` dla ustawienia indeksu na `timestamp`.

In [1]:
import pandas as pd

df = pd.read_csv("pomiary.csv")
df = df.sort_values("timestamp")
df = df.set_index("timestamp")
df = df.drop("index", axis=1)
df.head()

,voltage,current,temperature,technician_first_name,technician_last_name,status
timestamp,,,,,,
2023-01-01 05:43:00,231.267702,1.245972,101.748627,Emily,Brown,broken
2023-01-01 10:21:00,240.932880,1.527754,65.513886,Michael,Johnson,working
2023-01-01 10:25:00,240.383591,1.778813,57.968485,Emily,Brown,working
2023-01-01 12:28:00,0.000000,0.000000,54.056483,John,Doe,turned off
2023-01-01 13:59:00,240.955130,1.225599,61.739875,John,Doe,working


2. **Połącz kolumny `technician_first_name` i `technician_last_name` w jedną kolumnę o nazwie `technician`.**


In [2]:
df["technician"] = df["technician_first_name"] + " " + df["technician_last_name"]
df = df.drop(labels=["technician_first_name", "technician_last_name"], axis=1)
df.head()

,voltage,current,temperature,status,technician
timestamp,,,,,
2023-01-01 05:43:00,231.267702,1.245972,101.748627,broken,Emily Brown
2023-01-01 10:21:00,240.932880,1.527754,65.513886,working,Michael Johnson
2023-01-01 10:25:00,240.383591,1.778813,57.968485,working,Emily Brown
2023-01-01 12:28:00,0.000000,0.000000,54.056483,turned off,John Doe
2023-01-01 13:59:00,240.955130,1.225599,61.739875,working,John Doe


3. **Przekształć kolumnę `status` na trzy nowe kolumny kategoryczne: `status_working`, `status_broken`, i `status_turned off`.**
   - *Wskazówka*: Skorzystaj z `pd.get_dummies()` lub przypisz wartości `1` lub `0` na podstawie `status`.


In [149]:
status_dummies = pd.get_dummies(df['status'], prefix='status')
df = pd.concat([df, status_dummies], axis=1)

df.head()

,voltage,current,temperature,status,technician,status_broken,status_turned off,status_working
timestamp,,,,,,,,
2023-01-01 05:43:00,231.267702,1.245972,101.748627,broken,Emily Brown,True,False,False
2023-01-01 10:21:00,240.932880,1.527754,65.513886,working,Michael Johnson,False,False,True
2023-01-01 10:25:00,240.383591,1.778813,57.968485,working,Emily Brown,False,False,True
2023-01-01 12:28:00,0.000000,0.000000,54.056483,turned off,John Doe,False,True,False
2023-01-01 13:59:00,240.955130,1.225599,61.739875,working,John Doe,False,False,True


4. **Usuń wartości odstające (outliers) z kolumn `current` i `voltage`. Wartości odstające to te, które są większe niż 95. percentyl.**
   - *Wskazówka*: możesz użyć metody `.quantile()` do określenia granicy.


In [150]:
current_95 = df["current"].quantile(0.95)
voltage_95 = df["voltage"].quantile(0.95)

df = df[(df['current'] <= current_95) & (df['voltage'] <= voltage_95)]

df.head()

,voltage,current,temperature,status,technician,status_broken,status_turned off,status_working
timestamp,,,,,,,,
2023-01-01 05:43:00,231.267702,1.245972,101.748627,broken,Emily Brown,True,False,False
2023-01-01 10:21:00,240.932880,1.527754,65.513886,working,Michael Johnson,False,False,True
2023-01-01 10:25:00,240.383591,1.778813,57.968485,working,Emily Brown,False,False,True
2023-01-01 12:28:00,0.000000,0.000000,54.056483,turned off,John Doe,False,True,False
2023-01-01 13:59:00,240.955130,1.225599,61.739875,working,John Doe,False,False,True


5. **Zastąp wartości odstające (outliers) w kolumnie `temperature` średnią temperaturą odpowiednią dla kategorii `status` (np. inną dla stanu `broken` i `working`).**
   - *Wskazówka*: Skorzystaj z grupowania przy pomocy `groupby()` na podstawie `status` i zastąp wartości odstające średnią z każdej grupy.


In [151]:
temperature_95 = df["temperature"].quantile(0.95)
mean_temps = df.groupby("status")["temperature"].mean()
print(mean_temps)

df["temperature"] = df["temperature"].where((df["temperature"] <= temperature_95) & (df["status"] == "broken"), mean_temps["broken"])
df["temperature"] = df["temperature"].where((df["temperature"] <= temperature_95) & (df["status"] == "turned off"), mean_temps["turned off"])
df["temperature"] = df["temperature"].where((df["temperature"] <= temperature_95) & (df["status"] == "working"), mean_temps["working"])

df.head()


status
broken        169.812762
turned off     63.516461
working       100.726006
Name: temperature, dtype: float64


,voltage,current,temperature,status,technician,status_broken,status_turned off,status_working
timestamp,,,,,,,,
2023-01-01 05:43:00,231.267702,1.245972,100.726006,broken,Emily Brown,True,False,False
2023-01-01 10:21:00,240.932880,1.527754,63.516461,working,Michael Johnson,False,False,True
2023-01-01 10:25:00,240.383591,1.778813,63.516461,working,Emily Brown,False,False,True
2023-01-01 12:28:00,0.000000,0.000000,100.726006,turned off,John Doe,False,True,False
2023-01-01 13:59:00,240.955130,1.225599,63.516461,working,John Doe,False,False,True


6. **Dodaj dwie nowe kolumny: `power`, którą obliczysz na podstawie kolumn `voltage` i `current`, oraz `Normalized_Power`.**
   - *Wskazówka*: `power` to iloczyn `voltage` i `current`. Aby znormalizować `power`, użyj wzoru:  

     $\text{Normalized} = \frac{\text{Value} - \text{Min}}{\text{Max - Min}}$


In [152]:
df["power"] = df["voltage"] * df["current"]

min_power = df["power"].min()
max_power = df["power"].max()

df["normalized_power"] = (df["power"] - min_power) / (max_power - min_power)

df.head()

,voltage,current,temperature,status,technician,status_broken,status_turned off,status_working,power,normalized_power
timestamp,,,,,,,,,,
2023-01-01 05:43:00,231.267702,1.245972,100.726006,broken,Emily Brown,True,False,False,288.153058,0.605357
2023-01-01 10:21:00,240.932880,1.527754,63.516461,working,Michael Johnson,False,False,True,368.086250,0.773282
2023-01-01 10:25:00,240.383591,1.778813,63.516461,working,Emily Brown,False,False,True,427.597402,0.898304
2023-01-01 12:28:00,0.000000,0.000000,100.726006,turned off,John Doe,False,True,False,0.000000,0.000000
2023-01-01 13:59:00,240.955130,1.225599,63.516461,working,John Doe,False,False,True,295.314321,0.620402


7. **Standaryzuj wartości kolumny `temperature`.**
   - *Wskazówka*: Standaryzację można wykonać, odejmując średnią kolumny `temperature` i dzieląc przez jej odchylenie standardowe:  
     $ \text{Standardized} = \frac{\text{Value} - \text{Mean}}{\text{Std}} $
     użyj w tym celu metody `.apply` wraz z funcją `lambda`.


In [153]:
temp_mean = df["temperature"].mean()
temp_std = df["temperature"].std()
df["temperature"] = df["temperature"].apply(lambda x: (x - temp_mean) / temp_std)

df.head()

,voltage,current,temperature,status,technician,status_broken,status_turned off,status_working,power,normalized_power
timestamp,,,,,,,,,,
2023-01-01 05:43:00,231.267702,1.245972,1.648031,broken,Emily Brown,True,False,False,288.153058,0.605357
2023-01-01 10:21:00,240.932880,1.527754,-0.604554,working,Michael Johnson,False,False,True,368.086250,0.773282
2023-01-01 10:25:00,240.383591,1.778813,-0.604554,working,Emily Brown,False,False,True,427.597402,0.898304
2023-01-01 12:28:00,0.000000,0.000000,1.648031,turned off,John Doe,False,True,False,0.000000,0.000000
2023-01-01 13:59:00,240.955130,1.225599,-0.604554,working,John Doe,False,False,True,295.314321,0.620402


8. **Zmień nazwy wszystkich kolumn, aby zaczynały się z wielkiej litery.**
   - *Wskazówka*: Użyj `rename()` z funkcją `str.title()`.


In [154]:
df = df.rename(str.title, axis='columns')

df.head()

,Voltage,Current,Temperature,Status,Technician,Status_Broken,Status_Turned Off,Status_Working,Power,Normalized_Power
timestamp,,,,,,,,,,
2023-01-01 05:43:00,231.267702,1.245972,1.648031,broken,Emily Brown,True,False,False,288.153058,0.605357
2023-01-01 10:21:00,240.932880,1.527754,-0.604554,working,Michael Johnson,False,False,True,368.086250,0.773282
2023-01-01 10:25:00,240.383591,1.778813,-0.604554,working,Emily Brown,False,False,True,427.597402,0.898304
2023-01-01 12:28:00,0.000000,0.000000,1.648031,turned off,John Doe,False,True,False,0.000000,0.000000
2023-01-01 13:59:00,240.955130,1.225599,-0.604554,working,John Doe,False,False,True,295.314321,0.620402



9. **Dodaj 0,1 do każdego rekordu `Current`, który wystąpił przed 10 stycznia.**
   - *Wskazówka*: Użyj `loc[]` oraz warunku daty, aby zaktualizować tylko wybrane wartości `Current`.


In [155]:
df.loc[df.index < "2023-01-10", "Current"] += 0.1
df.head()

,Voltage,Current,Temperature,Status,Technician,Status_Broken,Status_Turned Off,Status_Working,Power,Normalized_Power
timestamp,,,,,,,,,,
2023-01-01 05:43:00,231.267702,1.345972,1.648031,broken,Emily Brown,True,False,False,288.153058,0.605357
2023-01-01 10:21:00,240.932880,1.627754,-0.604554,working,Michael Johnson,False,False,True,368.086250,0.773282
2023-01-01 10:25:00,240.383591,1.878813,-0.604554,working,Emily Brown,False,False,True,427.597402,0.898304
2023-01-01 12:28:00,0.000000,0.100000,1.648031,turned off,John Doe,False,True,False,0.000000,0.000000
2023-01-01 13:59:00,240.955130,1.325599,-0.604554,working,John Doe,False,False,True,295.314321,0.620402
